# CNTK 209 Part A: Berkeley Segmentation Dataset loader

## Preparing the training set

<p>This notebook serves as data preparation for the CNTK 209 tutorial. The aim of this tutorial is to showcase how we can use tools from CNTK to train models which will perform image super-resolution. The goal of the <b>single image super-resolution (SISR)</b> problem is to upscale a given image by some factor while keeping as much image details as possible and not making the image blurry. This problem is described in more detail in the remaining notebooks of this tutorial.</p>
<p>It is recommended for the user to complete the tutorial CNTK 206 before this. Some basic familiarity with deep convolutional networks is also welcome.</p>
<p>We will be using <b>Berkeley Segmetation Dataset (BSDS)</b>. It contains 300 images which we will download and then prepare for training of the models we will use. Image dimensions are 481 x 321 and 321 x 481.</p>

In [1]:
# Import the relevant modules to be used later
import urllib
import re
import os
import numpy as np
from PIL import Image

try:
    from urllib.request import urlretrieve, urlopen
except ImportError: 
    from urllib import urlretrieve, urlopen

## Data download
The data is not present in a .zip, .gz or in a similar packet. It is located in a folder on this <a href="https://www2.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/segbench/BSDS300/html/images/plain/normal/color/">link</a>. Therefore, we will use regular expression to find all image names and download them one by one into destination folder.

In [2]:
def download_data(images_dir, link):
    if not os.path.exists(images_dir):
        os.makedirs(images_dir)
    
    images_html = urlopen(link).read().decode('utf-8')
    
    #looking for .jpg images whose names are numbers
    image_regex = "[0-9]+.jpg"
    
    #remove duplicates
    image_list = set(re.findall(image_regex, images_html))
    print("Starting download...")
    
    for image in image_list:
        filename = os.path.join(images_dir, image)
        if not os.path.isfile(filename):
            urlretrieve(link + image, filename)
        else:
            print("File already exists", filename)
    
    print("Images available at: ", images_dir)

<p>Now we only need to call this function with appropriate parameters. This might take a few minutes.</p>

In [3]:
# Save the dataset (prefer our default path for the data)
data_dir = os.path.join("..", "Examples", "Image", "DataSets", "BerkeleySegmentationDataset")
if not os.path.exists(data_dir):
    data_dir = os.path.join("Data", "BerkeleySegmentationDataset")

#folder for raw images, before preprocess
images_dir = os.path.join(data_dir, "Images")

#link to BSDS dataset
link = "https://www2.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/segbench/BSDS300/html/images/plain/normal/color/"

download_data(images_dir, link)

Starting download...
Images available at:  Data/BerkeleySegmentationDataset/Images


## Data preparation
<p>This dataset contains only 300 images which is not enough for super-resolution training. The idea is to split images into 64 x 64 patches which will augment the training data. Function <code>prep_64</code> does exactly that.</p>
<p>Once we select some 64 x 64 patch, we downscale it by a factor of 2 and then upscale it by a factor of 2 again using bicubic interpolation. This will give us a blurry version of the original patch. In some approaches in the tutorial, the idea will be to learn the model which turns blurry pathes into clear ones. We will put blurry patches into one folder and normal patches into another. They will serve as minibatch sources for training. We will also sample a few test images here.</p>
<p>After processing each patch, we move 42 pixels down/right (so there is some overlapping) as long as we can. This can take a few minutes.</p>

In [4]:
#extract 64 x 64 patches from BSDS dataset
def prep_64(images_dir, patch_h, patch_w, train64_lr, train64_hr, tests):
    if not os.path.exists(train64_lr):
        os.makedirs(train64_lr)
    
    if not os.path.exists(train64_hr):
        os.makedirs(train64_hr)
    
    if not os.path.exists(tests):
        os.makedirs(tests)
    
    k = 0
    num = 0
    
    print("Creating 64 x 64 training patches and tests from:", images_dir)
    
    for entry in os.listdir(images_dir):
        filename = os.path.join(images_dir, entry)
        img = Image.open(filename)
        rect = np.array(img)
    
        num = num + 1
        
        if num % 50 == 0:
            img.save(os.path.join(tests, str(num) + ".jpg"))
            continue
    
        x = 0
        y = 0
    
        while(y + patch_h <= img.width):
            x = 0
            while(x + patch_w <= img.height):
                patch = rect[x : x + patch_h, y : y + patch_w]
                img_hr = Image.fromarray(patch, 'RGB')
                
                img_lr = img_hr.resize((patch_w // 2, patch_h // 2), Image.ANTIALIAS)
                img_lr = img_lr.resize((patch_w, patch_h), Image.BICUBIC)
                
                out_hr = os.path.join(train64_hr, str(k) + ".jpg")
                out_lr = os.path.join(train64_lr, str(k) + ".jpg")
                
                k = k + 1
                
                img_hr.save(out_hr)
                img_lr.save(out_lr)
                
                x = x + 42
            y = y + 42
    print("Done!")

<p>Since CNTK VGG19 model, which we will also download here and later use in training of <b>SRGAN</b> model, operates strictly on 224 x 224 images, we will need to prepare training data for models which turn 112 x 112 images into 224 x 224 images. We will train two such models (<b>SRResNet</b> and <b>SRGAN</b>) in the later part of this tutorial.</p>
<p>We use similar reasoning for augmenting the training data here with <code>prep_224</code>. We will be selecting 224 x 224 patches and downscaling them to 112 x 112 patches. The only difference is that now we also rotate patches to additionally increase the number of training samples because we don't get as much 224 x 224 patches as 64 x 64 patches. Like before, 224 x 224 patches go into one folder and 112 x 112 patches go into another.</p>

In [5]:
#extract 224 x 224 and 112 x 112 patches from BSDS dataset
def prep_224(images_dir, patch_h, patch_w, train112, train224):
    if not os.path.exists(train112):
        os.makedirs(train112)
    
    if not os.path.exists(train224):
        os.makedirs(train224)

    k = 0
    
    print("Creating 224 x 224 and 112 x 112 training patches from:", images_dir)
    
    for entry in os.listdir(images_dir):
        filename = os.path.join(images_dir, entry)
        img = Image.open(filename)
        rect = np.array(img)
    
        x = 0
        y = 0
    
        while(y + patch_h <= img.width):
            x = 0
            while(x + patch_w <= img.height):
                patch = rect[x : x + patch_h, y : y + patch_w]
                img_hr = Image.fromarray(patch, 'RGB')
                
                img_lr = img_hr.resize((patch_w // 2, patch_h // 2), Image.ANTIALIAS)
                
                for i in range(4):
                    out_hr = os.path.join(train224, str(k) + ".jpg")
                    out_lr = os.path.join(train112, str(k) + ".jpg")
                
                    k = k + 1
                
                    img_hr.save(out_hr)
                    img_lr.save(out_lr)
                
                    img_hr = img_hr.transpose(Image.ROTATE_90)
                    img_lr = img_lr.transpose(Image.ROTATE_90)
                
                x = x + 64
            y = y + 64
    print("Done!")

<p>We take the images we downloaded and run <code>prep</code> functions with adequate parameters. In the end, in the folder <b>Images</b> we will have a little bit less than 300 images from Berkeley Segmentation Dataset, depending on how many go to test set.</p>
<p>In the folder <b>train64_LR</b> we will have around 20000 64 x 64 blurry image patches that will be used for training. Their original counterparts will be located in the folder <b>train64_HR</b>.</p>
<p>In the folder <b>train224</b> we will have around 12000 original 224 x 224 patches. Their downscaled 112 x 112 counterparts will be located <b>train112</b>. Images that can later be used for testing will be in <b>tests</b>.</p>

In [6]:
#blurry 64x64 destination
train64_lr = os.path.join(data_dir, "train64_LR")

#original 64x64 destination
train64_hr = os.path.join(data_dir, "train64_HR")

#112x112 patches destination
train112 = os.path.join(data_dir, "train112")

#224x224 pathes destination
train224 = os.path.join(data_dir, "train224")

#tests destination
tests = os.path.join(data_dir, "tests")

#prep
prep_64(images_dir, 64, 64, train64_lr, train64_hr, tests)
prep_224(images_dir, 224, 224, train112, train224)

Creating 64 x 64 training patches and tests from: Data/BerkeleySegmentationDataset/Images
Done!
Creating 224 x 224 and 112 x 112 training patches from: Data/BerkeleySegmentationDataset/Images
Done!


<p>Now we are ready to start the training or we can just experiment with the evaluation of the pretrained models.</p>